In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <vector>
#include <torch/torch.h>

# 1 create tensor by picking a factory function in libtorch


## 1.1 using factory function

In [3]:
/* Factory Functions
    A factory function is a function that produces a new tensor. There are many factory functions available in PyTorch (both in Python and C++), which differ in the way they initialize a new tensor before returning it. All factory functions adhere to the following general “schema”:
    torch::<function-name>(<function-specific-options>, <sizes>, <tensor-options>)
*/

In [4]:
torch::Tensor tensor =  torch::rand({2,3});
std::cout << tensor << std::endl;

// <sizes> is an object of type IntArrayRef and specifies the shape of the resulting tensor,
std::cout << "tensor size: " << tensor.sizes() << std::endl;

 0.1155  0.6917  0.6978
 0.0185  0.4893  0.9475
[ CPUFloatType{2,3} ]
tensor size: [2, 3]


In [5]:
std::cout << tensor.sizes()[0];

2

In [6]:
std::cout << tensor.size(0);

2

In [7]:
// 1.1.2 Passing Function-Specific Parameters
torch::Tensor tensor_int = torch::randint(/*high=*/10, {2, 3});
std::cout << tensor_int << std::endl;

 5  6  9
 0  1  9
[ CPUFloatType{2,3} ]


## 1.2 create tensor from existing data
* torch::from_blob function

* torch::tensor function

In [8]:
std::vector<float> data_vector1d = {1,2,3,4};
std::cout << "data vector 1d: \n" << data_vector1d << std::endl;

std::vector<std::vector<float>> data_vector2d = {{1,2,3,4},{2,3,4,5}};
std::cout << "data vector 2d: \n" << data_vector2d << std::endl;

data vector 1d: 
1 2 3 4
data vector 2d: 
1 2 3 4 2 3 4 5


In [9]:
torch::Tensor tensor_from_vector1d = torch::from_blob(data_vector1d.data(), {2,2});
std::cout << "tensor from vector1d: \n" << tensor_from_vector1d << std::endl;

tensor from vector1d: 
 1  2
 3  4
[ CPUFloatType{2,2} ]


In [10]:
// error
torch::Tensor tensor_from_vector2d = torch::from_blob(data_vector2d.data(), {2,4});
std::cout << "tensor from vector2d: \n" << tensor_from_vector2d << std::endl;

tensor from vector2d: 
 1.8436e+30  3.0943e-41  1.8436e+30  3.0943e-41
 1.8436e+30  3.0943e-41  5.6779e+33  3.0943e-41
[ CPUFloatType{2,4} ]


In [11]:
/*
torch::tensor create function

*/
torch::Tensor tensor_from_vector1d_ = torch::tensor(data_vector1d);
std::cout << "tensor from vector1d: \n" << tensor_from_vector1d << std::endl;

tensor from vector1d: 
 1  2
 3  4
[ CPUFloatType{2,2} ]


In [12]:
// error
//torch::Tensor tensor_from_vector2d = torch::tensor(data_vector2d);
//std::cout << "tensor from vector: \n" << tensor_from_vector2d << std::endl;

# 2 simple operation to tensor
* member function of tensor
* in-built function in libtorch

In [13]:
torch::Tensor a = torch::rand({2,3});

In [14]:
std::cout << a << std::endl;

 0.9889  0.0063  0.2544
 0.6940  0.4358  0.4989
[ CPUFloatType{2,3} ]


In [15]:
auto b = a.reshape({3,2});
std::cout << b << std::endl;

 0.9889  0.0063
 0.2544  0.6940
 0.4358  0.4989
[ CPUFloatType{3,2} ]


In [16]:
auto c = a.pow(2);
std::cout << c << std::endl;

 0.9779  0.0000  0.0647
 0.4817  0.1899  0.2489
[ CPUFloatType{2,3} ]


In [17]:
auto d = a.abs();
std::cout << d << std::endl;

 0.9889  0.0063  0.2544
 0.6940  0.4358  0.4989
[ CPUFloatType{2,3} ]


In [18]:
auto e = torch::abs(a);
std::cout << e << std::endl;

 0.9889  0.0063  0.2544
 0.6940  0.4358  0.4989
[ CPUFloatType{2,3} ]


# 3 explore properties of tensor

In [19]:
std::cout << tensor_from_vector1d.sizes() << std::endl;

[2, 2]


In [20]:
std::cout << tensor_from_vector1d.dtype() << std::endl;

float


In [21]:
std::cout << tensor_from_vector1d.device() << std::endl;

cpu


In [22]:
std::cout << tensor_from_vector1d.layout() << std::endl;

Strided


In [23]:
std::cout << tensor_from_vector1d.requires_grad() << std::endl;

0


# 4 configure properties of the tensor

The previous section discussed function-specific arguments. Function-specific arguments can only change the values with which tensors are filled, and sometimes the size of the tensor. They never change things like the data type (e.g. float32 or int64) of the tensor being created, or whether it lives in CPU or GPU memory. The specification of these properties is left to the very last argument to every factory function: a TensorOptions object, discussed below.
TensorOptions is a class that encapsulates the construction axes of a Tensor. With construction axis we mean a particular property of a Tensor that can be configured before its construction (and sometimes changed afterwards). These construction axes are:
The dtype (previously “scalar type”), which controls the data type of the elements stored in the tensor,
The layout, which is either strided (dense) or sparse,
The device, which represents a compute device on which a tensor is stored (like a CPU or CUDA GPU),
The requires_grad boolean to enable or disable gradient recording for a tensor,
If you are used to PyTorch in Python, these axes will sound very familiar. The allowed values for these axes at the moment are:
For dtype: kUInt8, kInt8, kInt16, kInt32, kInt64, kFloat32 and kFloat64,
For layout: kStrided and kSparse,
For device: Either kCPU, or kCUDA (which accepts an optional device index),
For requires_grad: either true or false.*/


## 4.1 use the general solution: torch::TensorOptions

In [24]:
 auto options = torch::TensorOptions()
	    .dtype(torch::kFloat32)
	    .layout(torch::kStrided)
	    .device(torch::kCPU)
	    .requires_grad(true);
torch::Tensor tensor_dtype = torch::full({3, 4}, /*value=*/12.3, options);
std::cout << "tensor from torch::full: \n" << tensor_dtype << std::endl;

assert(tensor_dtype.dtype() == torch::kFloat32);
assert(tensor_dtype.layout() == torch::kStrided);
assert(tensor_dtype.device().type() == torch::kCPU); // or device().is_cuda()
assert(tensor_dtype.requires_grad());

tensor from torch::full: 
 12.3000  12.3000  12.3000  12.3000
 12.3000  12.3000  12.3000  12.3000
 12.3000  12.3000  12.3000  12.3000
[ CPUFloatType{3,4} ]


## 4.2 equivalent operation regarding tensor property

 Now, you may be thinking: do I really need to specify each axis for every new tensor I create?
       Fortunately, the answer is “no”, as every axis has a default value. These defaults are:
       kFloat32 for the dtype,
       kStrided for the layout,
       kCPU for the device,
       false for requires_grad.
    What this means is that any axis you omit during the construction of a TensorOptions object will take on its default value. For example, this is our previous TensorOptions object, but with the dtype and layout defaulted:
    in fact, we can even omit all axes to get an entirely defaulted TensorOptions object:
    auto options = torch::TensorOptions(); // or `torch::TensorOptions options;`
    A nice consequence of this is that the TensorOptions object we just spoke so much about can be entirely omitted from any tensor factory call:
    A 32-bit float, strided, CPU tensor that does not require a gradient.
    torch::Tensor tensor = torch::randn({3, 4});
    torch::Tensor range = torch::arange(5, 10);


In [25]:
// equivalent operation
torch::ones(10, torch::TensorOptions().dtype(torch::kFloat32));
torch::ones(10, torch::dtype(torch::kFloat32));
//and further instead of
torch::ones(10, torch::TensorOptions().dtype(torch::kFloat32).layout(torch::kStrided));
//we can just write
torch::ones(10, torch::dtype(torch::kFloat32).layout(torch::kStrided));
// use default property
// python syntax: torch.randn(3, 4, dtype=torch.float32, device=torch.device('cuda', 1), requires_grad=True)

@0x564279666fe0

## 4.3 properties conversion


Just as we can use TensorOptions to configure how new tensors should be created, we can also use TensorOptions to convert a tensor from one set of properties to a new set of properties. Such a conversion usually creates a new tensor and does not occur in-place. For example, if we have a source_tensor created with


In [26]:
torch::Tensor source_tensor = torch::randn({2, 3});

//we can convert it from int64 to float32:
torch::Tensor float_tensor = source_tensor.to(torch::kFloat32);

// The result of the conversion, float_tensor, is a new tensor pointing to new memory, unrelated to the source source_tensor.
// We can then move it from CPU memory to GPU memory:
    // if (torch::cuda::is_available()){
    //    torch::Tensor gpu_tensor = float_tensor.to(torch::kCUDA);
    //}

# 5 Copy of torch::Tensor and Tensor as argument in function

In [27]:
torch::Tensor x = torch::ones({2,3});
std::cout << x << std::endl;

 1  1  1
 1  1  1
[ CPUFloatType{2,3} ]


In [28]:
torch::Tensor y = x;
std::cout << y << std::endl;
y = y+0.2;
std::cout << y << std::endl;
std::cout << x << std::endl;

 1  1  1
 1  1  1
[ CPUFloatType{2,3} ]
 1.2000  1.2000  1.2000
 1.2000  1.2000  1.2000
[ CPUFloatType{2,3} ]
 1  1  1
 1  1  1
[ CPUFloatType{2,3} ]


In [29]:
y.data().fill_(0.3);
std::cout << y << std::endl;
std::cout << x << std::endl;

 0.3000  0.3000  0.3000
 0.3000  0.3000  0.3000
[ CPUFloatType{2,3} ]
 1  1  1
 1  1  1
[ CPUFloatType{2,3} ]


In [30]:
torch::Tensor z = x.clone();
z = z+0.02;
std::cout << z << std::endl;
std::cout << x << std::endl;

 1.0200  1.0200  1.0200
 1.0200  1.0200  1.0200
[ CPUFloatType{2,3} ]
 1  1  1
 1  1  1
[ CPUFloatType{2,3} ]


In [31]:
torch::Tensor w = torch::tensor({1.0, 2.0});
std::cout << w << std::endl;

 1
 2
[ CPUFloatType{2} ]


In [32]:
torch::Tensor ww = w;
std::cout << ww << std::endl;

 1
 2
[ CPUFloatType{2} ]


In [33]:
ww.data().fill_(0.1);
std::cout << ww << std::endl;
std::cout << w << std::endl;

 0.1000
 0.1000
[ CPUFloatType{2} ]
 0.1000
 0.1000
[ CPUFloatType{2} ]


In [34]:
torch::Tensor m = torch::tensor({1.0, 2.0});
std::cout << m << std::endl;

 1
 2
[ CPUFloatType{2} ]


In [35]:
torch::Tensor mm = m.clone();
std::cout << mm << std::endl;
mm.data().fill_(0.1);
std::cout << mm << std::endl;
std::cout << m << std::endl;

 1
 2
[ CPUFloatType{2} ]
 0.1000
 0.1000
[ CPUFloatType{2} ]
 1
 2
[ CPUFloatType{2} ]


In [36]:
torch::Tensor tensor_pass_by_value(torch::Tensor x){
    y = x+0.1;
    return y;
};

In [37]:
torch::Tensor a = torch::tensor({1.0, 2.0});
std::cout << a << std::endl;
torch::Tensor b = tensor_pass_by_value(a);
std::cout << b << std::endl;
std::cout << a << std::endl;

 1
 2
[ CPUFloatType{2} ]
 1.1000
 2.1000
[ CPUFloatType{2} ]
 1
 2
[ CPUFloatType{2} ]


In [38]:
torch::Tensor tensor_pass_by_value_(torch::Tensor x){
    x.data().fill_(0.33);
    return x;
};

In [39]:
torch::Tensor a = torch::tensor({1.0, 2.0});
std::cout << a << std::endl;
torch::Tensor b = tensor_pass_by_value_(a);
std::cout << b << std::endl;
std::cout << a << std::endl;

 1
 2
[ CPUFloatType{2} ]
 0.3300
 0.3300
[ CPUFloatType{2} ]
 0.3300
 0.3300
[ CPUFloatType{2} ]


## appendix 1: the type of tensor.size()

https://github.com/pytorch/pytorch/blob/master/c10/util/ArrayRef.h

In [40]:
torch::Tensor a_tensor = torch::randn({2,3,4});
std::cout << a_tensor << std::endl;

(1,.,.) = 
 -0.2116 -1.9207 -0.7819 -0.0965
  0.6448  0.2803 -0.6104 -0.2530
  0.3948  0.4878  1.9908  0.9813

(2,.,.) = 
 -0.2562 -1.5020 -1.0282  0.6078
 -0.7117 -0.2597 -0.7668  0.8546
  1.2169 -1.2268 -1.2287 -0.3314
[ CPUFloatType{2,3,4} ]


In [41]:
at::IntArrayRef shape_of_tensor = a_tensor.sizes();

In [42]:
std::cout << shape_of_tensor << std::endl;

[2, 3, 4]


In [43]:
c10::ArrayRef<int64_t> low_level_size_of_tensor = a_tensor.sizes();

In [44]:
std::cout << low_level_size_of_tensor << std::endl;

[2, 3, 4]


* at::IntArrayRef is a template specialization of c10::ArrayRef<>
~~~
using IntArrayRef = ArrayRef<int64_t>;
~~~

## appendix 2: torch::from_blob for the nd data input
> you need convert the nd data into 1d, then use the torch::from_blob

In [46]:
std::vector<int64_t> a = {1,0,1,1,0,0};
torch::Tensor a_ts = torch::from_blob(a.data(), {2,3}, torch::TensorOptions().dtype(torch::kInt64)).clone();
std::cout << a_ts << std::endl;

 1  0  1
 1  0  0
[ CPULongType{2,3} ]


In [47]:
std::vector<std::vector<int64_t>> d = {{0,1,0},{1,0,0}};

In [48]:
torch::Tensor d_error_ts = torch::from_blob(d.data(), {2,3}, torch::TensorOptions().dtype(torch::kInt64)).clone();
std::cout << d_error_ts << std::endl;

 9.4844e+13  9.4844e+13  9.4844e+13
 9.4843e+13  9.4843e+13  9.4843e+13
[ CPULongType{2,3} ]


In [49]:
template<typename T>
std::vector<T> onelinevector(const std::vector<std::vector<T>>& invector) {
        std::vector<T> v1d;
        if (invector.size() == 0)
            return v1d;
        v1d.reserve(invector.size() * invector.front().size());

        for (auto &innervector : invector) {
            v1d.insert(v1d.end(), innervector.begin(), innervector.end());
        }
        return v1d;
}

In [50]:
std::vector<int64_t> one_line_d = onelinevector(d);

In [51]:
torch::Tensor d_correct_ts = torch::from_blob(one_line_d.data(), {2,3}, torch::TensorOptions().dtype(torch::kInt64)).clone();
std::cout << d_correct_ts << std::endl;

 0  1  0
 1  0  0
[ CPULongType{2,3} ]
